### Basic library imports

In [14]:
import os
import pandas as pd
import concurrent.futures
import requests
import shutil
import subprocess

# Define the dataset folder
DATASET_FOLDER = '../dataset/'

# Load the datasets
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))

# Run the external Python scripts
subprocess.run(["python", "sanity.py", "--test_filename", os.path.join(DATASET_FOLDER, "sample_test.csv"),
                "--output_filename", os.path.join(DATASET_FOLDER, "sample_test_out.csv")])

subprocess.run(["python", "sanity.py", "--test_filename", os.path.join(DATASET_FOLDER, "sample_test.csv"),
                "--output_filename", os.path.join(DATASET_FOLDER, "sample_test_out_fail.csv")])

# Function to download images
def download_image(image_url, output_folder):
    try:
        img_data = requests.get(image_url, timeout=10).content
        file_name = os.path.join(output_folder, image_url.split('/')[-1])
        with open(file_name, 'wb') as handler:
            handler.write(img_data)
    except Exception as e:
        print(f"Error downloading {image_url}: {e}")

def download_images(image_urls, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Download images in parallel using ThreadPoolExecutor
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(lambda url: download_image(url, output_folder), image_urls)

# Example usage
image_urls = sample_test['image_link'].tolist()
download_images(image_urls, '../images')

# Assert that at least one image was downloaded
assert len(os.listdir('../images')) > 0, "No images were downloaded!"

# Remove the images folder if needed
shutil.rmtree('../images')
